In [4]:
import pythreejs as three
import json
import os
import ipywidgets
import ipywidgets.embed as embed
from IPython.display import display
from ipywidgets import VBox
import socket
import threading
import numpy as np
from pythreejs import Mesh, BufferGeometry, MeshStandardMaterial, PerspectiveCamera, Scene, AmbientLight, DirectionalLight, OrbitControls, Renderer, Matrix4
import trimesh
from trimesh.exchange import gltf
import time

class LanderVisualization:
    def __init__(self, model_paths):
        # Set up the scenes for inside and outside models.
        self.inside_scene = Scene(children=[AmbientLight(color='#ffffff')])
        self.outside_scene = Scene(children=[AmbientLight(color='#ffffff')])
        
        # Set up the camera.
        self.camera = PerspectiveCamera(position=[10, 10, 10], 
                                        children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.6)])
        
        # Set up controls.
        self.controls = OrbitControls(controlling=self.camera)
        
        # Set up renderers for both scenes.
        self.inside_renderer = Renderer(scene=self.inside_scene, 
                                        camera=self.camera, 
                                        controls=[self.controls], 
                                        antialias=True, 
                                        alpha=True, 
                                        width=400, 
                                        height=600)
        
        self.outside_renderer = Renderer(scene=self.outside_scene, 
                                         camera=self.camera, 
                                         controls=[self.controls], 
                                         antialias=True, 
                                         alpha=True, 
                                         width=400, 
                                         height=600)
        
        # Load the models.
        self.load_model(model_paths[0], self.inside_scene)
        self.load_model(model_paths[1], self.outside_scene)
        
        # Set up threading for receiving IMU data
        self.stop_thread = False
        self.thread = threading.Thread(target=self.simulate_imu_data)
        self.thread.start()

    def load_model(self, model_path, scene):
        """Loads a GLB model using trimesh and adds it to the specified scene."""
        mesh = trimesh.load(model_path, force='mesh')

        # Ensure the vertices and faces are properly formatted
        vertices = mesh.vertices.astype(np.float32)  # Convert vertices to float32 if not already
        faces = mesh.faces.flatten().astype(np.uint32)  # Convert faces to unsigned 32-bit integer

        geometry = BufferGeometry(attributes={
            'position': three.BufferAttribute(vertices, normalized=False),
            'index': three.BufferAttribute(faces, normalized=False)
        })

        material = MeshStandardMaterial(color='#808080')
        model = Mesh(geometry, material)
        scene.add(model)

    def display(self):
        """Displays the 3D models in a Jupyter notebook."""
        display(VBox([self.inside_renderer, self.outside_renderer]))
        
    def save_html(self, file_name):
        """Saves the current visualization as an HTML file."""
        box = VBox([self.inside_renderer, self.outside_renderer])
        with open(file_name, 'w') as f:
            embed.embed_minimal_html(f, views=[box], title='Lander Visualization')
    
    def simulate_imu_data(self):
        """Simulates IMU data to update the model's orientation."""
        while not self.stop_thread:
            # Generate a simulated quaternion representing rotation
            angle = time.time() % (2 * np.pi)  # Simple rotating angle
            quaternion = [np.cos(angle / 2), np.sin(angle / 2), 0, 0]  # Rotate around the x-axis
            self.update_orientation(quaternion)
            time.sleep(0.1)  # Update every 100ms

    def update_orientation(self, quaternion):
        """Updates the orientation of the models based on the received quaternion."""
        q = np.array(quaternion)
        # Convert quaternion to a rotation matrix
        rot_matrix = self.quaternion_to_rotation_matrix(q)
        matrix = Matrix4(elements=rot_matrix.flatten().tolist())
        matrix.matrixAutoUpdate = False
        
        # Update both models
        if hasattr(self, 'inside_scene') and hasattr(self, 'outside_scene'):
            for obj in self.inside_scene.children + self.outside_scene.children:
                if isinstance(obj, Mesh):
                    obj.matrix = matrix

    @staticmethod
    def quaternion_to_rotation_matrix(q):
        """Converts a quaternion into a 4x4 rotation matrix."""
        qw, qx, qy, qz = q
        rot_matrix = np.array([
            [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw, 0],
            [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw, 0],
            [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2, 0],
            [0, 0, 0, 1]
        ])
        return rot_matrix

    def close(self):
        """Stops the thread and closes resources."""
        self.stop_thread = True
        self.thread.join()


# Example usage
if __name__ == "__main__":
    print("Hi, I'm the lander visualization!")
    model_paths = [
        "C:/Users/Owner/Documents/CPEG/Code/asteria/cad/Asteria_Inside_Body.glb",
        "C:/Users/Owner/Documents/CPEG/Code/asteria/cad/Asteria_Outside_Body.glb"
    ]
    for model_path in model_paths:
        if not os.path.exists(model_path):
            print(f"Error: File not found at {model_path}")
    else:
        viz = LanderVisualization(model_paths)
        viz.display()


Hi, I'm the lander visualization!


Exception in thread Thread-6 (simulate_imu_data):
Traceback (most recent call last):
  File "C:\Users\Owner\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\Owner\Documents\CPEG\Code\asteria\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Owner\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_5608\440757769.py", line 88, in simulate_imu_data
    self.update_orientation(quaternion)
    ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_5608\440757769.py", line 103, in update_orientation
    obj.matrix = matrix
    ^^^^^^^^^^
  File "c:\Users\Owner\Documents\CPEG\Code\asteria\.venv\Lib\site-pa